In [1]:
import tensorflow as tf
import pandas as pd
import xarray as xr
import pickle
import numpy as np
from sklearn.preprocessing import StandardScaler

# Deep learning model
from tensorflow.keras.layers import Input, Dense, Reshape, Concatenate, Embedding
from tensorflow.keras.models import Model, load_model, Sequential
from keras.callbacks import EarlyStopping

2024-02-28 12:15:11.123239: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-28 12:15:11.123285: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-28 12:15:11.124647: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-28 12:15:11.132678: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-28 12:15:12.281492: W tensorflow/comp

In [2]:
metar_data = pd.read_csv('../00_data_preparation/metar/metar_dataset_with_airport_location.csv')
training_data = pd.read_csv('../01_data_analyzation/file_height_2021_01.csv')
validation_data = pd.read_csv('../01_data_analyzation/file_height_2021_02.csv')
testing_data = pd.read_csv('../01_data_analyzation/file_height_2021_03.csv')

In [3]:
training_data['contain_FZ'] = 1
validation_data['contain_FZ'] = 1
testing_data['contain_FZ'] = 1

In [4]:
y_train = training_data['contain_FZ']
y_test = testing_data['contain_FZ']

In [5]:
wind_speed = pd.read_csv('/lustre/storeB/project/AI/course/ens_wind_speed/ens_windspeed_testing.csv')

In [6]:
wind_speed

,site,sid,u10,v10,ws10,mslp,z500,tcc,sshf,slhf,cape,doy,month,ws_obs
0,SN10380,0,-2.162065,0.233785,2.174668,100012.921875,51198.382812,1.000000,1.117032e+06,1.453334e+05,2.537766,4,1,2.2
1,SN10380,0,-0.795285,2.281249,2.415900,102589.992188,54399.980469,0.999673,5.027644e+06,2.625227e+05,0.000000,11,1,1.6
2,SN10380,0,-2.948594,6.015121,6.698947,100186.062500,51931.156250,1.000000,2.798910e+06,-4.458621e+05,0.004156,15,1,11.8
3,SN10380,0,-1.717631,3.669883,4.051950,99371.578125,49946.523438,0.992263,2.040646e+06,-1.602110e+06,0.098823,18,1,7.2
4,SN10380,0,-0.697049,2.044248,2.159821,101522.320312,51519.312500,0.890855,1.649672e+06,3.623278e+05,2.661640,22,1,0.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10199,SN9580,79,-0.248234,2.006799,2.022094,101491.000000,54169.445312,0.999826,-9.756760e+04,2.246795e+05,2.635664,329,11,1.9
10200,SN9580,79,0.419636,3.742002,3.765458,98695.046875,50504.589844,0.999113,3.018947e+06,6.981211e+05,6.853627,339,12,3.8
10201,SN9580,79,-0.404330,5.008265,5.024560,98928.179688,50739.281250,0.952109,4.895494e+06,1.010430e+05,2.775676,346,12,3.2
10202,SN9580,79,0.182370,1.006346,1.022737,99886.890625,51171.753906,0.944745,1.092381e+06,-1.038893e+05,0.854998,350,12,2.5


In [7]:
training_data

,airport,time,air_temperature_0m,fog_area_fraction,surface_air_pressure,air_pressure_at_sea_level,contain_FZ
0,ENAL,2021-01-11 00:00:00,278.706785,0.012861,100168.817661,100205.954471,1
1,ENAL,2021-01-11 02:00:00,278.709869,0.072659,100076.139400,100110.627146,1
2,ENAL,2021-01-11 03:00:00,278.592989,0.000387,99975.716528,100013.231258,1
3,ENAL,2021-01-11 04:00:00,278.687528,0.038568,99939.028688,99979.091311,1
4,ENAL,2021-01-11 23:00:00,278.772482,0.036937,99511.671265,99547.256479,1
...,...,...,...,...,...,...,...
8659,ENZV,2021-01-26 22:00:00,272.207328,0.999463,100771.402555,101008.002839,1
8660,ENZV,2021-01-26 23:00:00,272.035219,1.000000,100759.374486,100995.048360,1
8661,ENZV,2021-01-27 00:00:00,270.759331,1.000000,100697.100937,100931.793600,1
8662,ENZV,2021-01-27 02:00:00,268.319824,0.657408,100666.828998,100901.627517,1


In [8]:
training_data['airport_id'] = pd.factorize(training_data['airport'])[0]
testing_data['airport_id'] = pd.factorize(testing_data['airport'])[0]

In [9]:
training_data

,airport,time,air_temperature_0m,fog_area_fraction,surface_air_pressure,air_pressure_at_sea_level,contain_FZ,airport_id
0,ENAL,2021-01-11 00:00:00,278.706785,0.012861,100168.817661,100205.954471,1,0
1,ENAL,2021-01-11 02:00:00,278.709869,0.072659,100076.139400,100110.627146,1,0
2,ENAL,2021-01-11 03:00:00,278.592989,0.000387,99975.716528,100013.231258,1,0
3,ENAL,2021-01-11 04:00:00,278.687528,0.038568,99939.028688,99979.091311,1,0
4,ENAL,2021-01-11 23:00:00,278.772482,0.036937,99511.671265,99547.256479,1,0
...,...,...,...,...,...,...,...,...
8659,ENZV,2021-01-26 22:00:00,272.207328,0.999463,100771.402555,101008.002839,1,37
8660,ENZV,2021-01-26 23:00:00,272.035219,1.000000,100759.374486,100995.048360,1,37
8661,ENZV,2021-01-27 00:00:00,270.759331,1.000000,100697.100937,100931.793600,1,37
8662,ENZV,2021-01-27 02:00:00,268.319824,0.657408,100666.828998,100901.627517,1,37


In [10]:
n_airports = len(training_data['airport_id'].unique())

In [11]:
input_aid = Input(shape=(1,))

In [12]:
output_dim = 5

In [13]:
embedding_layer = Embedding(input_dim=n_airports, output_dim=output_dim, name="embedding_airport")(input_aid)

2024-02-28 12:15:19.079081: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [14]:
embedding_layer

<KerasTensor: shape=(None, 1, 5) dtype=float32 (created by layer 'embedding_airport')>

In [15]:
emb_cat = Reshape(target_shape=(output_dim, ))(embedding_layer)

In [16]:
emb_cat

<KerasTensor: shape=(None, 5) dtype=float32 (created by layer 'reshape')>

In [17]:
data_params = ['air_temperature_0m', 'fog_area_fraction', 'surface_air_pressure', 'air_pressure_at_sea_level']

In [18]:
scaler = StandardScaler()

In [19]:
training_data[data_params] = scaler.fit_transform(training_data[data_params])

In [20]:
training_data

,airport,time,air_temperature_0m,fog_area_fraction,surface_air_pressure,air_pressure_at_sea_level,contain_FZ,airport_id
0,ENAL,2021-01-11 00:00:00,1.032538,-0.001098,0.128671,-0.386368,1,0
1,ENAL,2021-01-11 02:00:00,1.032953,0.746596,0.083799,-0.450602,1,0
2,ENAL,2021-01-11 03:00:00,1.017247,-0.157074,0.035178,-0.516230,1,0
3,ENAL,2021-01-11 04:00:00,1.029951,0.320340,0.017415,-0.539234,1,0
4,ENAL,2021-01-11 23:00:00,1.041366,0.299944,-0.189498,-0.830215,1,0
...,...,...,...,...,...,...,...,...
8659,ENZV,2021-01-26 22:00:00,0.159178,12.335201,0.420423,0.154071,1,37
8660,ENZV,2021-01-26 23:00:00,0.136051,12.341912,0.414600,0.145342,1,37
8661,ENZV,2021-01-27 00:00:00,-0.035396,12.341912,0.384449,0.102720,1,37
8662,ENZV,2021-01-27 02:00:00,-0.363203,8.058203,0.369792,0.082393,1,37


## Model

In [21]:
param_features_dim = Input(shape=(4,))
dense_layer_1 = Dense(units=32, activation='relu')(param_features_dim)
dense_layer_2 = Dense(units=16, activation='relu')(dense_layer_1)

In [22]:
# Concatenate embedded categorical airport feature with param features
concat = Concatenate()([emb_cat, dense_layer_2])

In [23]:
output_layer = Dense(units=1, activation='sigmoid')(concat)

In [24]:
output_layer

<KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'dense_2')>

In [25]:
model = Model(inputs=[input_aid, param_features_dim], outputs=output_layer)

In [26]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 1)]                  0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 4)]                  0         []                            
                                                                                                  
 embedding_airport (Embeddi  (None, 1, 5)                 190       ['input_1[0][0]']             
 ng)                                                                                              
                                                                                                  
 dense (Dense)               (None, 32)                   160       ['input_2[0][0]']         

In [27]:
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [28]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 1)]                  0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 4)]                  0         []                            
                                                                                                  
 embedding_airport (Embeddi  (None, 1, 5)                 190       ['input_1[0][0]']             
 ng)                                                                                              
                                                                                                  
 dense (Dense)               (None, 32)                   160       ['input_2[0][0]']         

In [29]:
model.fit([training_data['airport_id'], training_data[data_params]], y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
217/217 [==============================] - 1s 2ms/step - loss: 0.2104 - accuracy: 0.9948 - val_loss: 0.0156 - val_accuracy: 1.0000
Epoch 2/10
217/217 [==============================] - 0s 1ms/step - loss: 0.0052 - accuracy: 1.0000 - val_loss: 0.0026 - val_accuracy: 1.0000
Epoch 3/10
217/217 [==============================] - 0s 1ms/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 0.0010 - val_accuracy: 1.0000
Epoch 4/10
217/217 [==============================] - 0s 1ms/step - loss: 5.7811e-04 - accuracy: 1.0000 - val_loss: 5.3780e-04 - val_accuracy: 1.0000
Epoch 5/10
217/217 [==============================] - 0s 1ms/step - loss: 3.2219e-04 - accuracy: 1.0000 - val_loss: 3.2730e-04 - val_accuracy: 1.0000
Epoch 6/10
217/217 [==============================] - 0s 1ms/step - loss: 2.0280e-04 - accuracy: 1.0000 - val_loss: 2.1738e-04 - val_accuracy: 1.0000
Epoch 7/10
217/217 [==============================] - 0s 1ms/step - loss: 1.3758e-04 - accuracy: 1.0000 - val_loss: 1.5344e-

In [30]:
# Make predictions
predictions = model.predict([testing_data['airport_id'], testing_data[data_params]])

151/151 [==============================] - 0s 719us/step


In [31]:
predictions

array([[1.],
       [1.],
       [1.],
       ...,
       [1.],
       [1.],
       [1.]], dtype=float32)